In [1]:
import pandas as pd
import random

# ✅ Paths to your original CSVs
single_csv_path = "clean_singlelabel_train.csv"         # 295k+ samples
multi_csv_path  = "mixed_multilabel_train.csv"          # 15k mixed samples

# ✅ Parameters
num_single_label = 14000
num_multi_label = 6000
random_seed = 42

# ✅ Load both CSVs
df_single = pd.read_csv(single_csv_path)
df_multi = pd.read_csv(multi_csv_path)

# ✅ Rename single-label column to 'labels'
if 'label' in df_single.columns:
    df_single['labels'] = df_single['label']
    df_single.drop(columns=['label'], inplace=True)

# ✅ Randomly sample
df_single_sampled = df_single.sample(n=num_single_label, random_state=random_seed).reset_index(drop=True)
df_multi_sampled = df_multi.sample(n=num_multi_label, random_state=random_seed).reset_index(drop=True)

# ✅ Combine
df_mixed = pd.concat([df_single_sampled, df_multi_sampled], ignore_index=True)
df_mixed = df_mixed.sample(frac=1, random_state=random_seed).reset_index(drop=True)  # shuffle

# ✅ Save
df_mixed.to_csv("train_split_mixed.csv", index=False)
print(f"✅ Mixed training CSV created with {len(df_mixed)} samples:")
print(f"   🐦  Single-label: {num_single_label}")
print(f"   🐦  Multi-label : {num_multi_label}")


✅ Mixed training CSV created with 20000 samples:
   🐦  Single-label: 14000
   🐦  Multi-label : 6000


In [3]:
import os
import shutil
import pandas as pd
from tqdm import tqdm

# ✅ CSV with relative filepaths
csv_path = "train_split_mixed.csv"
df = pd.read_csv(csv_path)

# ✅ Source roots
single_root = "E:/birdclef-2024/spectrograms"
multi_root  = "E:/birdclef-2024/mixed_multilabel_spects"

# ✅ Destination root
target_dir = "E:/birdclef-2024/train_mixed_spects"
os.makedirs(target_dir, exist_ok=True)

# ✅ Start copy
print(f"📦 Copying {len(df)} spectrograms to {target_dir}...\n")

for filepath in tqdm(df["filepath"], desc="🔄 Copying files"):
    fname = os.path.basename(filepath)

    # Detect source
    if "mixed_multilabel_spects" in filepath or fname.startswith("mix_"):
        src = os.path.join(multi_root, fname)
    else:
        # species subfolder in original
        species = os.path.basename(os.path.dirname(filepath))
        src = os.path.join(single_root, species, fname)

    dst = os.path.join(target_dir, fname)
    shutil.copy(src, dst)

print("✅ All spectrograms copied successfully.")


📦 Copying 20000 spectrograms to E:/birdclef-2024/train_mixed_spects...



🔄 Copying files: 100%|██████████| 20000/20000 [02:32<00:00, 131.02it/s]

✅ All spectrograms copied successfully.


In [5]:
import pandas as pd
import os

csv_path = "train_split_mixed.csv"
output_path = "train_split_mixed_updated.csv"
target_dir = "E:/birdclef-2024/train_mixed_spects"

df = pd.read_csv(csv_path)

# Replace with new path
df["filepath"] = df["filepath"].apply(lambda x: os.path.join(target_dir, os.path.basename(x)))

# Save updated CSV
df.to_csv(output_path, index=False)
print(f"✅ Updated CSV saved as {output_path}")


✅ Updated CSV saved as train_split_mixed_updated.csv


In [25]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import os

class BirdMultiLabelDataset(Dataset):
    def __init__(self, csv_file, image_size=(300, 300), transform=None):
        self.df = pd.read_csv(csv_file)
        self.image_size = image_size
        self.transform = transform

        self.df['labels'] = self.df['labels'].apply(lambda x: x.split("|"))
        self.mlb = MultiLabelBinarizer()
        self.label_matrix = self.mlb.fit_transform(self.df['labels'])
        self.label_names = self.mlb.classes_

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['filepath']).convert("RGB").resize(self.image_size)

        if self.transform:
            image = self.transform(image)
        else:
            image = torch.tensor(np.array(image) / 255.0).permute(2, 0, 1).float()

        label = torch.tensor(self.label_matrix[idx]).float()
        return image, label


In [27]:
from torchvision import transforms
from torch.utils.data import DataLoader

image_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

dataset = BirdMultiLabelDataset("train_split_mixed_updated.csv", transform=image_transforms)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [29]:
train_dataset = BirdMultiLabelDataset("train_split_mixed_train.csv", transform=image_transforms)
val_dataset = BirdMultiLabelDataset("train_split_mixed_val.csv", transform=image_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [35]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("train_split_mixed_updated.csv")
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.to_csv("train_split_mixed_train.csv", index=False)
val_df.to_csv("train_split_mixed_val.csv", index=False)


In [37]:
import timm
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model("efficientnet_b3", pretrained=True, num_classes=181)
model.to(device)

# Binary label matrix from your dataset
label_counts = dataset.label_matrix.sum(axis=0)  # shape (181,)
total_samples = len(dataset)

# Inverse frequency → rarer classes get higher weights
pos_weights = torch.tensor((total_samples - label_counts) / (label_counts + 1e-6), dtype=torch.float32).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [39]:
def apply_label_smoothing(labels, smoothing=0.05):
    return labels * (1 - smoothing) + 0.5 * smoothing


In [41]:
from sklearn.metrics import f1_score
import numpy as np
import torch

def evaluate_model(model, val_loader, device, threshold=0.5):
    model.eval()
    preds, true = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            output = torch.sigmoid(model(x)).cpu().numpy()
            preds.append(output)
            true.append(y.cpu().numpy())

    preds = np.vstack(preds)
    true = np.vstack(true)

    preds_bin = (preds > threshold).astype(int)
    val_f1 = f1_score(true, preds_bin, average='macro')

    return val_f1


In [ ]:
from sklearn.metrics import f1_score
from tqdm import tqdm

best_val_f1 = 0.0

for epoch in range(6):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{6}"):
        images, labels = images.to(device), labels.to(device)
        smoothed = apply_label_smoothing(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, smoothed)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1} Training Loss: {avg_loss:.4f}")

    # ✅ Optional: Validation step if you have val_loader
    model.eval()
    preds, true = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            output = torch.sigmoid(model(x)).cpu().numpy()
            preds.append(output)
            true.append(y.cpu().numpy())

    preds = np.vstack(preds)
    true = np.vstack(true)

    # Convert to binary (threshold at 0.5)
    preds_bin = (preds > 0.5).astype(int)

    # Macro F1-score
    val_f1 = f1_score(true, preds_bin, average='macro')
    print(f"✅ Epoch {epoch+1} Val Macro F1: {val_f1:.4f}")

    # ✅ Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_multilabel_model.pth")
        print(f"💾 Saved model with F1: {val_f1:.4f}")


Epoch 1/6: 100%|██████████| 500/500 [43:58<00:00,  5.28s/it]


📘 Epoch 1 Training Loss: 2.5783
✅ Epoch 1 Val Macro F1: 0.0182
💾 Saved model with F1: 0.0182


Epoch 2/6: 100%|██████████| 500/500 [42:53<00:00,  5.15s/it]


📘 Epoch 2 Training Loss: 2.4793
✅ Epoch 2 Val Macro F1: 0.0195
💾 Saved model with F1: 0.0195


Epoch 3/6:  75%|███████▌  | 376/500 [30:43<10:28,  5.07s/it]